# Final Project: Earth Analytics Python Course, Spring 2020
## Steph Shepherd & Lauren Herwehe

In [5]:
# Import libraries
import warnings
from glob import glob
import os

import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import geopandas as gpd

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

### Psuedocode
Overlay hydropower dam locations and rivers (with additional attributes on degree of regulations) on top of protected areas to identify which and to what degree protected areas are impacted by proposed hydropower dams

* Open each shapefile X
* Check CRS of shapefile and make them the same
* Crop shapefiles if we choose to focus on a specific area (don’t think we’re doing this)
* Fill NA values
* Grab the desired attributes from each shapefile
* Add the attributes to a pandas dataframe
* Remove duplicates in WDPA sites, and remove Ramsar sites from it since we have a separate file with a more comprehensive list of them
* Create an IF statement that tags overlap of ramsar/WDPA sites with Grand Dams
* Create functions in code as needed

In [9]:
# Open the necessary shapefiles with geopandas
protected_areas = gpd.read_file(os.path.join("final-project-data", "wdpa-data", "wdpa-boundaries", "WDPA_Apr2020-shapefile-polygons.shp"))
ramsar_areas = gpd.read_file(os.path.join("final-project-data", "ramsar-data", "ramsar-boundaries", "features_publishedPolygon.shp"))
dam_locs = gpd.read_file(os.path.join("final-project-data", "grand-data", "grand-dams", "GRanD_dams_v1_3.shp"))

# Check the crs of the files
print(protected_areas.crs)
print(ramsar_areas.crs)
print(dam_locs.crs)

# # Clean the no data values in each shapefile
# sjer_roads_cl["RTTYP"] = sjer_roads_cl["RTTYP"].fillna("Unknown")
# sjer_roads_cl = sjer_roads_cl[~sjer_roads_cl.is_empty]

epsg:4326
epsg:4326
epsg:4326


In [ ]:
# Open the necessary files with geopandas
ca_counties = gpd.read_file(os.path.join("data", "spatial-vector-lidar",
                                         "california", "CA_Counties", "CA_Counties_TIGER2016.shp"))
roads = gpd.read_file(os.path.join("data", "spatial-vector-lidar", "global", "ne_10m_roads",
                                   "ne_10m_roads.shp"))

# # Check the crs of the files
# print(ca_counties.crs)
# print(roads.crs)

# Reproject both layers to epsg 0570
ca_counties_5070 = ca_counties.to_crs(epsg=5070)
roads_5070 = roads.to_crs(epsg=5070)

# Select only the three counties of interest
three_counties = ca_counties_5070[ca_counties_5070['NAME'].isin(
    ["Siskiyou", "Modoc", "Del Norte"])]

# # Check the new crs of the files
# print(three_counties_5070.crs)
# print(roads_5070.crs)

# Clip the roads data using the clip_shp module
roads_5070_cl = cl.clip_shp(roads_5070, three_counties)

# Redefine the CRS of the roads layer
roads_5070_cl.crs = three_counties.crs

# Assign the roads to their respective county with a spatial join
roads_region = gpd.sjoin(roads_5070_cl, three_counties,
                         how="inner", op='intersects')

In [ ]:
# PLOT 1 - Place only the code required to create a plot of your data here
# Additional processing code can go above this code cell

# Settting color palettes and sizing for roads and points
pointsPalette = {'trees': 'chartreuse',
                 'grass': 'darkgreen', 'soil': 'burlywood'}

roadPalette = {'M': 'grey', 'S': "blue",
               'C': "magenta", 'Unknown': "lightgrey"}

lineWidths = {'M': .5, 'S': 2, 'C': 2, 'Unknown': .5}

# Create figure
fig, ax = plt.subplots(figsize=(10, 10))

for ctype, data in sjer_plots.groupby('plot_type'):
    color = pointsPalette[ctype]
    label = ctype
    data.plot(color=color,
              ax=ax,
              label=label,
              markersize=100)

for ctype, data in sjer_roads_cl.groupby('RTTYP'):
    color = roadPalette[ctype]
    label = ctype
    data.plot(color=color,
              ax=ax,
              linewidth=lineWidths[ctype],
              label=label)

ax.set(title='Madera County Roads and Study Plot Locations')

ax.legend(fontsize=15,
          frameon=False,
          loc=('lower right'),
          title="LEGEND")

ax.set_axis_off()
plt.axis('equal')
### DO NOT REMOVE LINE BELOW ###
plot01_roads_plot_locs = nb.convert_axes(plt, which_axes="current")

In [ ]:
# PLOT 2 - Place only the code required to plot your data here
# Additional processing code can go above this code cell
# Important: name your final geodataframe for county boundaries: three_counties

# Plot the data
fig, ax = plt.subplots(figsize=(10, 5))
three_counties.plot(edgecolor="black",
                    facecolor='none',
                    ax=ax)

roads_region.plot(column='NAME',
                  ax=ax,
                  legend=True)

ax.set(title='California Roads in Del Norte, Modoc, and Siskiyou Counties')
ax.set_axis_off()
plt.axis('equal')

### DO NOT REMOVE LINE BELOW ###
plot02_county_roads_clip = nb.convert_axes(plt, which_axes="current")

In [ ]:
# TABLE 1 - Place the code required to create the dataframe
# Important: name your final geodataframe: cali_roads_summary

# # Calculate the total length of road in each county
roads_region['length'] = roads_region.length
cali_roads_summary = roads_region[['length', 'NAME']].groupby('NAME').sum()

# Print the new table
print(cali_roads_summary)

In [ ]:
# Download the data
data = et.data.get_data(
    url=' https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip')

# Open the necessary files with geopandas
countries = gpd.read_file(os.path.join("data", "earthpy-downloads",
                                       "ne_10m_admin_0_countries", "ne_10m_admin_0_countries.shp"))

# Subset the data
pop_data = countries[["REGION_WB", "CONTINENT",
                      "POP_RANK", "POP_EST", 'geometry']]

# Dissolve and aggregate the data
mean_region_val = pop_data.dissolve(by='REGION_WB', aggfunc=['sum', 'mean'])

# Getting column names to use in making our plots
list(mean_region_val.columns)

In [ ]:
# PLOT 3 - Place only the code required to plot your data here
# Additional processing code can go above this code cell
# Important: name your final geodataframe: mean_region_val
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

mean_region_val.plot(column=('POP_EST', 'sum'),
                     legend=True,
                     cmap='OrRd',
                     ax=ax1)

mean_region_val.plot(column=('POP_RANK', 'mean'),
                     cmap='OrRd',
                     legend=True,
                     ax=ax2)

plt.suptitle('Global Total Estimated Population by Region', fontsize=16)

plt.show()


### DO NOT REMOVE LINE BELOW ###
plot04_global_population = nb.convert_axes(plt, which_axes="all")